In [1]:
import numpy as np
import matplotlib.pyplot as plt
from velocity_functions import *
from scipy.integrate import odeint
# %matplotlib inline

radius  = {'RBC':3.75e-6,'WBC':6.25e-6,'plate':1.19e-6,'bac':2.5e-6} # radius in meters
density = {'RBC':1093,   'WBC':1066,   'plate':1053,   'bac': 1095 } # density in kg/m3
sedcoef = {'RBC':12e-7,  'WBC':1.2e-7, 'plate':0.032e-7} # Sedimentation coefficient in s (Van Wie)
viscosity = {'plas':0.0026} # viscosity of plasma in kg/m s

# Velocity and Integration schemes
The current scheme integrates a 1D scalar conservation law using zero flux B.C. at both walls.
I want to put in stokes velocity and the right dimensions
### Next steps:
1. non-dimensionalize code for time, flux, and velocity. I will follow Lerche 2001's non-dimensionalization (__Complete__)
$$x^{*} = \frac{x}{L}$$
$$T = \frac{L}{U_{0}}$$
$$t^{*} = \frac{tU_{0}}{L} = \frac{t}{T}$$
$$u^{*} = \frac{u}{u_{0}}$$  
  
2. clean velocity functions
  1. use tuples, not dictionary for values
3. write multicomponent sedimentation functions
  1. MLB
  2. _bulk_ flux based on RBC (Michaels), bacteria and others based on RZ for total solids concentration
  3. porosity
4. Find how to do multicomponent Godunov

In [2]:
def get_c(α,r,cell_params,mode='RZ'): #r is a stand-in value
    '''return non-dimensional velocity'''
    RPM,D,density = cell_params
    
    # Evaluate scalar stokes velocity as f(RPM,r,D)
    c_max = stokes(RPM,r,D,density)
    hsc = 1.0
    
    if mode == "RZ":
        hsc = (1-α)**2 # Richardson-zaki correlation
        
    return hsc

def get_q(α_cells,*u0):
    '''
    returns the speed and volumetric flux of j particle types
    inputs: jxn array (checkdims for consistency) of j particle concentrations.
    
    number of species and stokes velocity should come from class attributes
    output: 
    '''
    # assign slip velocity of cells relative to fluid using HSC
    c_cells_slip = get_c(np.sum(α_cells,axis=0)) # sum the particle concentrations. Replace with polydisperse HSC.
    c_cells_slip = np.vstack((c_cells_slip,c_cells_slip))
    
    
    # calculate plasma α, initialize array of plasma speeds (0), and combine with cell values
    α_plasma = 1-np.sum(α_cells,axis=0)
    c_plasma = np.zeros(ngrd)
    α,c_slip = [np.vstack((species,fluid)) for species,fluid in zip((α_cells,c_cells_slip),(α_plasma,c_plasma))]
    
    # compute absolute velocities (and flux) using Davis and Gecol 1994 equation 2
    c_avg_slip = np.sum(c_slip*α,axis=0)
    c_abs = c_slip - c_avg_slip
    q_abs = α*c_abs
    
    return q_abs,c_abs

def bc(inside,left_edge,right_edge):
    output       = np.zeros(ngrd+2)
    output[0]    = left_edge
    output[-1]   = right_edge
    output[1:-1] = inside
    return output

def godunov(α,t,r,cell_params):
    α_bc       = np.empty(ngrd+2)
    α_bc[0]    = 0.0
    α_bc[-1]   = 1.0
    α_bc[1:-1] = α

    r_bc       = np.empty(ngrd+2)
    r_bc[0]    = r[0] -Δx # be careful taking global variables!
    r_bc[-1]   = r[-1]+Δx
    r_bc[1:-1] = r
    
#     evaluate fluxes at each cell and initialize cell interfaces
    q = α_bc*get_c(α_bc,r_bc,cell_params) # I may need r_bc to be dimensional or not.
#     q = get_q(α_bc) It won't be that simple to go from scalar integration to system integration.
    qf = np.zeros(ngrdf)
    
    # apply Godunov flux at each cell interface
    for i in range(ngrdf):
        if α_bc[i] >= α_bc[i+1]:
            qf[i] = max(q[i],q[i+1])
        elif α_bc[i] <= 0 and 0 <= α_bc[i+1]:
            qf[i] = 0
        else:
            qf[i] = min(q[i],q[i+1])
    
    # apply upwinding scheme with the godunov interface fluxes
    dαdt = 1/Δx*(qf[:-1]-qf[1:])
    return dαdt

In [3]:
# Experimental parameters
α_rbc_0 = 0.4
RPM = 3000
spin = (RPM,radius['RBC'],density['RBC'])

tend   = 1      # s
ntimes = 50   
ngrd   = 50
r1     = 0.057  # m
r2     = 0.06   # m
L      = r2-r1  # m

# Non-dimensionalized variables
U0 = stokes(RPM,r2,radius['RBC'],density['RBC'])
T  = L/U0 # seconds it takes to traverse length going at max speed.
cycles = tend/T

# Time and space grids                  Dimensions
times  = np.linspace(0,cycles,ntimes) # 
Δt     = tend/ntimes                  # s
ngrdf  = ngrd+1                       #
xf     = np.linspace(r1/r2,1,ngrdf)   # 
Δx     = xf[1]-xf[0]                  # 
x      = (xf[1:] + xf[:-1])/2         #

α0  = np.ones(ngrd)*α_rbc_0
αall,diag = odeint(godunov, α0, times,args=(x,spin),full_output=1)

TypeError: len() of unsized object

In [ ]:
L,U0,T,tend/T

In [ ]:
cfl = Δt*get_c(0.35,r1,spin)/Δx; cfl

In [ ]:
# %matplotlib inline
fig = plt.figure()
fig.figsize=(10,10)

color = plt.get_cmap('Reds')
colors = iter(color(np.linspace(0,1,ntimes)))
for j in range(ntimes):
    plt.plot(x,αall[j,:],color=next(colors))

plt.xlabel('position')
plt.ylabel(r'$\alpha$')
plt.xlim([r1/r2,1]);
plt.show()

### Animations

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML, Image
rc('animation',html='html5')
%matplotlib inline

fig, ax = plt.subplots()
ax.set_xlim([r1/r2,1])
ax.set_ylim([0,1])
line, = ax.plot([],[])

def init():
    line.set_data([],[])
    ax.set_xlabel('Normalized Distance')
    ax.set_ylabel('Hematocrit')
    return (line,)
def animate(i):
    line.set_data(x,αall[i,:])
    line.set_color('red')
    return (line,)
anim = animation.FuncAnimation(fig,animate,init_func=init,
                               frames=ntimes,interval=Δt*1000,blit=True)

anim
# anim.save('wave_simulation.mp4')